# Pre-Requisites

In [ ]:
!pip3 install -U sentence-transformers

In [ ]:
!pip3 install pinecone-client pandas

## Query Generation

In [1]:
import pandas as pd

query_passage_df = pd.read_csv('datasets/query_passage.csv')
query_passage_df.head()

,Unnamed: 0,query,passage
0,0,what does ocd mean,One of the most helpful things in my own recov...
1,1,is it good to talk about an ocd,One of the most helpful things in my own recov...
2,2,does ocd really help recovery,One of the most helpful things in my own recov...
3,3,what is being a positive person,There is something powerful about knowing that...
4,4,who is an inspirational person,There is something powerful about knowing that...


## Negative Mining

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-tas-b')
model.max_seq_length = 256

model


/home/rezamohideen/myProjects/semantic-search/ENV/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [3]:
import pinecone

with open('secrets', 'r') as fp:
    API_KEY = fp.read()  # get api key app.pinecone.io

pinecone.init(
    api_key=API_KEY,
    environment='us-west1-gcp'
)
# create a new genq index if does not already exist
if 'negative-mine' not in pinecone.list_indexes():
    pinecone.create_index(
        'negative-mine',
        dimension=model.get_sentence_embedding_dimension(),
        metric='dotproduct',
        pods=1  # increase for faster mining
    )
# connect
index = pinecone.Index('negative-mine')

In [4]:
def get_pairs(df):
    pairs = []
    for index, row in df.iterrows():
        pairs.append((row['query'], row['passage']))

    return pairs

In [ ]:
# upload to pinecoe DO NOT RUN

from tqdm import tqdm
import numpy as np

pairs_gen = get_pairs(query_passage_df)  # generator that loads (query, passage) pairs
blogs = pd.read_csv('datasets/blogs.csv')
blogs = blogs.replace({np.nan: None})


pairs = []
to_upsert = []
passage_batch = []
id_batch = []
metadata_batch = []
batch_size = 64  # encode and upload size

for i, (query, passage) in enumerate(tqdm(pairs_gen)):
    pairs.append((query, passage))
    # we do this to avoid passage duplication in the vector DB
    if passage not in passage_batch: 
        passage_batch.append(passage)
        id_batch.append(str(i))

        metadata = blogs[blogs['text'] == passage]
        par = metadata.iloc[0]['paragraph'] if metadata.iloc[0]['paragraph'] else ''
        article = metadata.iloc[0]['article'] if metadata.iloc[0]['article'] else ''
        
        metadata_batch.append({'paragraph': par, 'article': article})
    # on reaching batch_size, we encode and upsert
    if len(passage_batch) == batch_size:
        embeds = model.encode(passage_batch).tolist()
        # upload to index
        index.upsert(vectors=list(zip(id_batch, embeds, metadata_batch)))
        # refresh batches
        passage_batch = []
        id_batch = []
        metadata_batch = []
        
# check number of vectors in the index
index.describe_index_stats()

### Test Base Model

In [6]:
import pinecone

with open('secrets', 'r') as fp:
    API_KEY = fp.read()  # get api key app.pinecone.io

pinecone.init(
    api_key=API_KEY,
    environment='us-west1-gcp'
)
index = pinecone.Index('negative-mine')

In [17]:
def convert_url(passage):
    sep = passage.split(',')
    start = sep[0].split(' ')[:4]
    end = sep[-1].split(' ')[-4:]

    start = '%20'.join(start)
    end = '%20'.join(end)

    return start + ',' + end

def convert_url_v2(passage):
    sep = passage.split(',')
    start = sep[0].split(' ')[:4]
    end = sep[-1].split(' ')[-4:]
    if len(start) < 2:
        end = sep[-1].split(' ')[-6:]

    start = '%20'.join(start)
    end = '%20'.join(end)

    return start + ',' + end

def query_db(query, model, index, passages):
    query_emb = model.encode(query, convert_to_tensor=True, show_progress_bar=False)
    res = index.query(query_emb.tolist(), top_k=10, include_metadata=True)

    nocd = 'https://www.treatmyocd.com'

    print(f'Search Query: {query}\n')
    print('---------------------------------------------------------------------------------------------------------------------')
    print('Results\n')
    for item in res.matches:
        print(f"Article: {nocd}{item['metadata']['article']}#:~:text={convert_url(passage_dict[int(item['id'])])}")
        print(f"Paragraph Header: {item['metadata']['paragraph']}")
        print(f"{item['score']} {passage_dict[int(item['id'])]}...\n")

In [19]:
pairs_gen = get_pairs(query_passage_df)
passage_dict = {i: p for i, (q, p) in enumerate(pairs_gen)}

from IPython.display import clear_output

while True:
    clear_output(wait=True)
    query = input("Search NOCD: ")
    if query == 'quit': break
    query_db(query=query, model=model, index=index, passages=passage_dict)

Search Query: pocd symptoms

---------------------------------------------------------------------------------------------------------------------
Results

Article: https://www.treatmyocd.com/blog/erp-is-effective-for-pocd#:~:text=For%20individuals%20with%20POCD,this%20might%20look%20like: 
Paragraph Header: 
107.378708 For individuals with POCD, this might look like: ...

Article: https://www.treatmyocd.com/blog/pocd-symtoms-and-treatment#:~:text=But%20another%20group%20responds,thoughts%20about%20children.%20
Paragraph Header: Pedophilia vs. Pedophilia OCD
100.898842 But another group responds very differently to these thoughts, which feel like a challenge to everything they believe they really are. They have a form of obsessive-compulsive disorder (OCD) referred to as pedophilia OCD, or POCD. Subtypes like POCD aren’t distinct conditions—they’re convenient ways of referring to a specific, common set of OCD symptoms. Like any subtype, POCD involves obsessions (intrusive, unwanted tho